In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

image_size = (320, 180)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory("data", validation_split=0.2, subset="training", seed=1337, image_size=image_size, batch_size=batch_size).prefetch(buffer_size=32)
test_ds = tf.keras.preprocessing.image_dataset_from_directory("data", validation_split=0.2, subset="validation", seed=1337, image_size=image_size, batch_size=batch_size).prefetch(buffer_size=32)


def make_model(input_shape):
    inputs = keras.Input(shape=input_shape)
    x = keras.Sequential(
        [
            layers.RandomFlip("horizontal"),
            layers.RandomRotation(0.1),
        ]
    )(inputs)

    x = layers.Rescaling(1.0 / 255)(x)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])
        previous_block_activation = x

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    activation = "sigmoid"
    units = 1

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)

model = make_model(input_shape=image_size + (3,))

epochs = 15

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=test_ds,
)



Found 2035 files belonging to 2 classes.
Using 1628 files for training.
Found 2035 files belonging to 2 classes.
Using 407 files for validation.
Epoch 1/15
51/51 [==============================] - 97s 2s/step - loss: 0.3133 - accuracy: 0.8649 - val_loss: 0.6843 - val_accuracy: 0.5553
Epoch 2/15
51/51 [==============================] - 94s 2s/step - loss: 0.1797 - accuracy: 0.9355 - val_loss: 0.6959 - val_accuracy: 0.5553
Epoch 3/15
51/51 [==============================] - 94s 2s/step - loss: 0.0852 - accuracy: 0.9705 - val_loss: 0.7160 - val_accuracy: 0.5553
Epoch 4/15
51/51 [==============================] - 95s 2s/step - loss: 0.0794 - accuracy: 0.9748 - val_loss: 0.7839 - val_accuracy: 0.5553
Epoch 5/15
51/51 [==============================] - 97s 2s/step - loss: 0.0564 - accuracy: 0.9853 - val_loss: 0.8251 - val_accuracy: 0.5553
Epoch 6/15
51/51 [==============================] - 111s 2s/step - loss: 0.0631 - accuracy: 0.9773 - val_loss: 0.8587 - val_accuracy: 0.5577
Epoch 7/15
51/

In [5]:
model.save("trained", save_format="tf")
# model.summary()


INFO:tensorflow:Assets written to: trained/assets
